## Fit ramberg osgood

In [39]:
import numpy as np
import paramaterial as pam
from paramaterial import DataSet, DataItem
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt

In [ ]:
ds = DataSet('data/02 trimmed small data', 'info/02 trimmed small info.xlsx')

In [ ]:
from pam.models import